In [1]:
using QuantEcon
using Interpolations
using Distributions

"""
Job Search with Permanent and Transient Components
Authors: Qingyin Ma and John Stachurski

transition function of the state process:
  w_t = eta_t + thet_t * xi_t,
  ln (thet_t) = ln (thet_{t-1}) + ln (u_t).

"""

type Search_Problem_SV
    bet::Float64  # the discount factor
    c0_tilde::Float64  # the unemployment compensation
    sig::Float64  # the coefficient of relative risk aversion
    rho::Float64  # the autoregressiion coefficient
    gam_u::Float64  # variance of the shock process w.r.t (theta_t) process
    gam_xi::Float64  # variance of the (xi_t) process
    mu_eta::Float64  # mean of the (eta_t) process
    gam_eta::Float64  # variance of the (eta_t) process
    thet_min::Float64  # minimum of the thet grid point
    thet_max::Float64  # maximum of the thet grid point
    thet_size::Int  # grid size of thet
    thet_grids::Vector{Float64}  # grid points of thet
    w_min::Float64  # munimum of the w grid point
    w_max::Float64  # maximum of the w grid point
    w_size::Int  # grid size of w
    w_grids::Vector{Float64}  # grid points of w
    mc_size::Int  # size of Monte Carlo draws
    draws::Vector{Float64}  # Monte carlo draws
    
    function Search_Problem_SV(;bet=.95,
                               c0_tilde=.6, 
                               sig=1., 
                               rho=.75, 
                               gam_u=1e-4,
                               gam_xi=5e-4, 
                               mu_eta=0., 
                               gam_eta=1e-6, 
                               thet_min=1e-3, 
                               thet_max=10., 
                               thet_size=200, 
                               thet_grids=Array{Float64}(100),
                               w_min=1e-3, 
                               w_max=10., 
                               w_size=100, 
                               w_grids=Array{Float64}(100), 
                               mc_size=1000, 
                               draws=Array{Float64}(1000))
        
        # making grid points for (thet, w)
        thet_grids = linspace(thet_min, thet_max, thet_size)
        w_grids = linspace(w_min, w_max, w_size)
        
        # making Monte Carlo samples
        draws = randn(mc_size)
        
        return new(bet, c0_tilde, sig, rho, gam_u, gam_xi, mu_eta, gam_eta, 
                   thet_min, thet_max, thet_size, thet_grids,
                   w_min, w_max, w_size, w_grids, mc_size, draws)
    end
end



# the CRRA utility function
function util_func(w, sigma)
    if sigma == 1.0
        uw = log.(w)
    else
        uw = w.^(1. - sigma) / (1. - sigma)
    end
    return uw
end



# =================== The Bellman Operator ==================== #
function Bellman_operator(sp::Search_Problem_SV, v)
    # simplify names
    bet, c0_tilde, sig = sp.bet, sp.c0_tilde, sp.sig
    rho, gam_u, gam_xi = sp.rho, sp.gam_u, sp.gam_xi
    mu_eta, gam_eta = sp.mu_eta, sp.gam_eta
    thet_size, thet_grids = sp.thet_size, sp.thet_grids
    w_size, w_grids = sp.w_size, sp.w_grids
    mc_size, draws = sp.mc_size, sp.draws
    
    # interpolate (and extrapolate) to obtain a function
    vf = extrapolate(interpolate((thet_grids, w_grids), v, 
                     Gridded(Linear())), Flat())
    
    c0 = util_func(c0_tilde, sig) # the flow continuation reward
    
    # create empty matrix to store T(v)
    new_v = Array{Float64}(thet_size, w_size)
    
    for j = 1:w_size
        w = w_grids[j]
        # the exit reward at grid point w_j
        v1 = util_func(w, sig) / (1. - bet)  
        
        # MC samples : eta'
        eta_draws = exp.(mu_eta + sqrt(gam_eta) * draws)
        # MC samples : xi'
        xi_draws = exp.(sqrt(gam_xi) * draws)
        
        for i = 1:thet_size
            thet = thet_grids[i] 
            
            # MC samples : thet'
            thet_draws = exp.(rho * log(thet) + sqrt(gam_u) * draws)
            # MC samples : w'
            w_draws = eta_draws + thet_draws .* xi_draws            
            # MC samples : v(thet', w')
            intg_draws = Array{Float64}(mc_size)
            for k = 1:mc_size
                intg_draws[k] = vf[thet_draws[k], w_draws[k]] 
            end
            
            # the continuation value at grid point (thet_i, w_j)
            v2 = c0 + bet * mean(intg_draws)
            # the value function at grid point (thet_i, w_j)
            new_v[i,j] = max(v1, v2)
        end
    end
    return new_v        
end



# ====================== The Jovanovic Operator ======================= #
function Jovanovic_operator(sp::Search_Problem_SV, psi)
    # simplify names
    bet, c0_tilde, sig = sp.bet, sp.c0_tilde, sp.sig
    rho, gam_u, gam_xi = sp.rho, sp.gam_u, sp.gam_xi
    mu_eta, gam_eta = sp.mu_eta, sp.gam_eta
    thet_size, thet_grids = sp.thet_size, sp.thet_grids
    w_size, w_grids = sp.w_size, sp.w_grids
    mc_size, draws = sp.mc_size, sp.draws
    
    # interpolate (and extrapolate) to obtain a function
    #psi_f = extrapolate(interpolate(thet_grids, psi,
    #                    Gridded(Linear())), Flat())
    psi_f = LinInterp(thet_grids, psi)
    
    c0 = util_func(c0_tilde, sig) # the flow continuation reward
    
    # MC samples : eta'
    eta_draws = exp.(mu_eta + sqrt(gam_eta) * draws)
    # MC samples : xi'
    xi_draws = exp.(sqrt(gam_xi) * draws)
   
    # create empty vector (thet_size * 1) to store Q (psi)
    new_psi = Array{Float64}(thet_size)
    
    for i = 1:thet_size
        # MC samples : thet'
        thet = thet_grids[i]
        thet_draws = exp.(rho * log(thet) + sqrt(gam_u) * draws)
        # MC samples : r(w'), the exit reward
        integ_1 = util_func(eta_draws + thet_draws .* xi_draws, sig) / (1. - bet)
        # MC samples : psi(thet')
        integ_2 = Array{Float64}(mc_size)
        for k = 1:mc_size
            integ_2[k] = psi_f(thet_draws[k])
        end        
        # MC samples : max{r(w'), psi(thet')}
        integ = max.(integ_1, integ_2)
        # the i-th element of the vector Q(psi)
        new_psi[i] = c0 + bet * mean(integ)
    end
    return new_psi
end

Jovanovic_operator (generic function with 1 method)

In [2]:
# Computation time of CVI : different grid sizes

thet_size_vals = [200, 200, 300, 300, 400, 400]  # the list of thet grid points
w_size_vals    = [200, 400, 200, 400, 200, 400]  # the list of w grid points

loops_cvi = 50  # the number of iterations performed for each case

# create empty matrix to store the time taken of CVI
time_cvi = Array{Float64}(loops_cvi, length(thet_size_vals))

# store the sig values used in each simulation
sig_vals = Array{Float64}(length(thet_size_vals))

for j=1:length(thet_size_vals)
    sp = Search_Problem_SV(sig=1., thet_size=thet_size_vals[j], 
                           w_size=w_size_vals[j])
    psi_0 = ones(sp.thet_size)  # initial guess
    
    for i=1:loops_cvi
        tic()  # start the clock
        psi_new = Jovanovic_operator(sp, psi_0)
        time_cvi[i,j] = toq()  # calculate the time elapsed
        psi_0 = psi_new
    end
    sig_vals[j] = sp.sig
    println("Loop $j finished ... ")
end

# key loops for which we want to calculate the time elapsed (CVI)
keyloops_cvi = [10, 20, 50]   
# create a vector to store the time elapsed (CVI) for the selected key loops
time_cvi_keyloops = Array{Float64}(length(keyloops_cvi), length(thet_size_vals))

for i=1:length(keyloops_cvi)
    for j=1:length(thet_size_vals)
        time_cvi_keyloops[i,j] = sum(time_cvi[1:keyloops_cvi[i], j])
    end
end

row_1 = time_cvi_keyloops[1,:]  # time taken : loop 10, all cases (grid sizes)
row_2 = time_cvi_keyloops[2,:]  # time taken : loop 20, all cases (grid sizes)
row_3 = time_cvi_keyloops[3,:]  # time taken : loop 50, all cases (grid sizes)


println("")
println("----------------------------------------------------------------")
println(" Computation time CVI : different grid sizes")
println("")
println(" Key parameters : sig = $sig_vals")
println("")
println(" thet_sizes  : $thet_size_vals")
println(" w_sizes     : $w_size_vals")
println("")
println(" Key loops : $keyloops_cvi")
println("")
println(" Time of CVI (column: grid sizes) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")


Loop 1 finished ... 
Loop 2 finished ... 
Loop 3 finished ... 
Loop 4 finished ... 
Loop 5 finished ... 
Loop 6 finished ... 

----------------------------------------------------------------
 Computation time CVI : different grid sizes

 Key parameters : sig = [1.0,1.0,1.0,1.0,1.0,1.0]

 thet_sizes  : [200,200,300,300,400,400]
 w_sizes     : [200,400,200,400,200,400]

 Key loops : [10,20,50]

 Time of CVI (column: grid sizes) : 

 Loop 10 : [0.325144,0.175799,0.259146,0.267567,0.360803,0.348147]
 Loop 20 : [0.493438,0.339471,0.516521,0.529421,0.726443,0.687851]
 Loop 50 : [1.01447,0.824252,1.27744,1.28883,1.78646,1.75673]

----------------------------------------------------------------



In [3]:
# Computation time of VFI : different grid sizes

thet_size_vals = [200, 200, 300, 300, 400, 400]  # the list of thet grid points
w_size_vals    = [200, 400, 200, 400, 200, 400]  # the list of w grid points

loops_vfi = 50  # the number of iterations performed for each case

# create empty matrix to store the time taken of VFI
time_vfi = Array{Float64}(loops_vfi, length(thet_size_vals))

# store the sig values used in each simulation
sig_vals = Array{Float64}(length(thet_size_vals))

for j=1:length(thet_size_vals)
    sp = Search_Problem_SV(sig=1., thet_size=thet_size_vals[j], 
                           w_size=w_size_vals[j])
    v_0 = ones(sp.thet_size, sp.w_size)  # initial guess (VFI)
    
    for i=1:loops_vfi
        tic()  # start the clock
        v_new = Bellman_operator(sp, v_0)
        time_vfi[i,j] = toq()  # calculate the time elapsed
        v_0 = v_new
    end
    sig_vals[j] = sp.sig
    println("Loop $j finished ... ")
end

# key loops for which we want to calculate the time elapsed (VFI)
keyloops_vfi = [10, 20, 50]   
# create a vector to store the time elapsed (VFI) for the selected key loops
time_vfi_keyloops = Array{Float64}(length(keyloops_vfi), length(thet_size_vals))

for i=1:length(keyloops_vfi)
    for j=1:length(thet_size_vals)
        time_vfi_keyloops[i,j] = sum(time_vfi[1:keyloops_vfi[i], j])
    end
end

row_1 = time_vfi_keyloops[1,:]  # time taken : loop 10, all cases (grid sizes)
row_2 = time_vfi_keyloops[2,:]  # time taken : loop 20, all cases (grid sizes)
row_3 = time_vfi_keyloops[3,:]  # time taken : loop 50, all cases (grid sizes)


println("")
println("----------------------------------------------------------------")
println(" Computation time VFI : different grid sizes")
println("")
println(" Key parameters : sig = $sig_vals")
println("")
println(" thet_sizes  : $thet_size_vals")
println(" w_sizes     : $w_size_vals")
println("")
println(" Key loops : $keyloops_vfi")
println("")
println(" Time of VFI (column: grid sizes) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 
Loop 2 finished ... 
Loop 3 finished ... 
Loop 4 finished ... 
Loop 5 finished ... 
Loop 6 finished ... 

----------------------------------------------------------------
 Computation time VFI : different grid sizes

 Key parameters : sig = [1.0,1.0,1.0,1.0,1.0,1.0]

 thet_sizes  : [200,200,300,300,400,400]
 w_sizes     : [200,400,200,400,200,400]

 Key loops : [10,20,50]

 Time of VFI (column: grid sizes) : 

 Loop 10 : [29.6959,62.4706,43.779,95.185,59.4696,128.727]
 Loop 20 : [58.3732,125.637,87.4384,190.224,118.748,257.321]
 Loop 50 : [144.375,314.3,218.615,475.783,297.343,644.57]

----------------------------------------------------------------



In [4]:
# Computation time of CVI : different parameter values 

sig_vals = [2., 2., 2., 3., 3., 3., 4., 4., 4.]  # the list of sig values
rho_vals = [.8, .7, .6, .8, .7, .6, .8, .7, .6]  # the list of rho values

loops_cvi = 50  # the number of iterations performed for each case

# create empty matrix to store the time taken of CVI
time_cvi = Array{Float64}(loops_cvi, length(sig_vals))

# create empty matrix to store the grid sizes used for simulation
grid_sizes = Array{Float64}(2)

for j=1:length(sig_vals)
    sp = Search_Problem_SV(sig=sig_vals[j], rho=rho_vals[j],
                           thet_size=300, w_size=300)
    psi_0 = ones(sp.thet_size)  # initial guess (CVI)
    
    for i=1:loops_cvi
        tic()  # start the clock
        psi_new = Jovanovic_operator(sp, psi_0)
        time_cvi[i,j] = toq()  # calculate the time elapsed
        psi_0 = psi_new
    end
    println("Loop $j finished ... ")
    # record the grid sizes used in simulation
    grid_sizes[1], grid_sizes[2] = sp.thet_size, sp.w_size
end

# key loops for which we want to calculate the time elapsed (CVI)
keyloops_cvi = [10, 20, 50]   
# create a vector to store the time elapsed (CVI) for the selected key loops
time_cvi_keyloops = Array{Float64}(length(keyloops_cvi), length(sig_vals))

for i=1:length(keyloops_cvi)
    for j=1:length(sig_vals)
        time_cvi_keyloops[i,j] = sum(time_cvi[1:keyloops_cvi[i], j])
    end
end

row_1 = time_cvi_keyloops[1,:]  # time taken : loop 10, all cases (grid sizes)
row_2 = time_cvi_keyloops[2,:]  # time taken : loop 20, all cases (grid sizes)
row_3 = time_cvi_keyloops[3,:]  # time taken : loop 50, all cases (grid sizes)


println("")
println("----------------------------------------------------------------")
println(" Computation time CVI : different parameter values")
println("")
println(" Key parameters : ")
println(" sig = $sig_vals")
println(" rho = $w_size_vals")
println("")
println(" Grid sizes (thet, w) : $grid_sizes")
println("")
println(" Key loops : $keyloops_cvi")
println("")
println(" Time of CVI (column: parameter values) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 
Loop 2 finished ... 
Loop 3 finished ... 
Loop 4 finished ... 
Loop 5 finished ... 
Loop 6 finished ... 
Loop 7 finished ... 
Loop 8 finished ... 
Loop 9 finished ... 

----------------------------------------------------------------
 Computation time CVI : different parameter values

 Key parameters : 
 sig = [2.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,4.0]
 rho = [200,400,200,400,200,400]

 Grid sizes (thet, w) : [300.0,300.0]

 Key loops : [10,20,50]

 Time of CVI (column: parameter values) : 

 Loop 10 : [0.204805,0.161196,0.162998,0.374573,0.393803,0.385736,0.372071,0.370614,0.366362]
 Loop 20 : [0.371016,0.320461,0.320402,0.745271,0.778026,0.764563,0.741216,0.737928,0.738878]
 Loop 50 : [0.866394,0.833779,0.794347,1.90772,1.89423,1.89477,1.85002,1.83665,1.83771]

----------------------------------------------------------------



In [5]:
# Computation time of VFI : different parameter values

sig_vals = [2., 2., 2., 3., 3., 3., 4., 4., 4.]  # the list of sig values
rho_vals = [.8, .7, .6, .8, .7, .6, .8, .7, .6]  # the list of rho values

loops_vfi = 50  # the number of iterations performed for each case

# create empty matrix to store the time taken of VFI
time_vfi = Array{Float64}(loops_vfi, length(sig_vals))

# create empty matrix to store the grid sizes used for simulation
grid_sizes = Array{Float64}(2)

for j=1:length(sig_vals)
    sp = Search_Problem_SV(sig=sig_vals[j], rho=rho_vals[j], 
                           thet_size=300, w_size=300)
    v_0 = ones(sp.thet_size, sp.w_size)  # initial guess (VFI)
    
    for i=1:loops_vfi
        tic()  # start the clock
        v_new = Bellman_operator(sp, v_0)
        time_vfi[i,j] = toq()  # calculate the time elapsed
        v_0 = v_new
    end
    println("Loop $j finished ... ")
    # record the grid sizes used in simulation
    grid_sizes[1], grid_sizes[2] = sp.thet_size, sp.w_size
end

# key loops for which we want to calculate the time elapsed (VFI)
keyloops_vfi = [10, 20, 50]   
# create a vector to store the time elapsed (VFI) for the selected key loops
time_vfi_keyloops = Array{Float64}(length(keyloops_vfi), length(sig_vals))

for i=1:length(keyloops_vfi)
    for j=1:length(sig_vals)
        time_vfi_keyloops[i,j] = sum(time_vfi[1:keyloops_vfi[i], j])
    end
end

row_1 = time_vfi_keyloops[1,:]  # time taken : loop 10, all cases (grid sizes)
row_2 = time_vfi_keyloops[2,:]  # time taken : loop 20, all cases (grid sizes)
row_3 = time_vfi_keyloops[3,:]  # time taken : loop 50, all cases (grid sizes)


println("")
println("----------------------------------------------------------------")
println(" Computation time VFI : different parameter values")
println("")
println(" Key parameters : ")
println(" sig = $sig_vals")
println(" rho = $rho_vals")
println("")
println(" Grid sizes (thet, w) : $grid_sizes")
println("")
println(" Key loops : $keyloops_vfi")
println("")
println(" Time of VFI (row: key loops, column: parameter values) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 
Loop 2 finished ... 
Loop 3 finished ... 
Loop 4 finished ... 
Loop 5 finished ... 
Loop 6 finished ... 
Loop 7 finished ... 
Loop 8 finished ... 
Loop 9 finished ... 

----------------------------------------------------------------
 Computation time VFI : different parameter values

 Key parameters : 
 sig = [2.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,4.0]
 rho = [0.8,0.7,0.6,0.8,0.7,0.6,0.8,0.7,0.6]

 Grid sizes (thet, w) : [300.0,300.0]

 Key loops : [10,20,50]

 Time of VFI (row: key loops, column: parameter values) : 

 Loop 10 : [69.7466,67.8647,66.6143,69.7952,67.6819,66.7036,69.9623,67.8375,66.4333]
 Loop 20 : [139.227,135.855,133.114,139.173,135.436,132.933,139.437,135.53,132.819]
 Loop 50 : [349.084,339.836,333.235,346.926,339.099,331.855,348.837,338.525,332.442]

----------------------------------------------------------------

